<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/%D0%BC%D0%BE%D1%8F_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_RAG_Retriaval_Augmented_Generation_%D0%B2%D0%B0%D1%80_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain

In [14]:
#Обработка текста, токенизация - деление на чанки
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt_tab')

text = "Считай по нашему мы выпили немного. Не вру ейбогу - скажи Серега. И если б водку бы не гнали из опилок чего б нам было с трех, четырех, пяти бутылок. Сначала пили близ прилавка в закуточки, но это были еще цветочки, но а когда коляска подкатила тогда у нас было по две на рыло."
chunks = sent_tokenize(text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [15]:
chunks

['Считай по нашему мы выпили немного.',
 'Не вру ейбогу - скажи Серега.',
 'И если б водку бы не гнали из опилок чего б нам было с трех, четырех, пяти бутылок.',
 'Сначала пили близ прилавка в закуточки, но это были еще цветочки, но а когда коляска подкатила тогда у нас было по две на рыло.']

In [16]:
#Генерация эмбеддингов через sentence_transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")   #paraphrase-multilingual-MiniLM-L12-v2
embeddings = model.encode(chunks)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
embeddings

array([[ 0.02307638,  0.04903948, -0.01733485, ...,  0.08246319,
         0.02260365, -0.07418048],
       [ 0.01886017,  0.15764475, -0.05373492, ...,  0.05890214,
        -0.01029115, -0.0478964 ],
       [ 0.00717446,  0.06206214,  0.02017272, ...,  0.03750903,
         0.05552401, -0.10624763],
       [-0.00758227,  0.0512202 ,  0.00443223, ...,  0.02030315,
        -0.00122372, -0.05814464]], dtype=float32)

In [26]:
embeddings.shape

(4, 384)

In [29]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.6 MB/s eta 0:00:00


In [32]:
#Создание векторной базы данных FAISS  (Chroma не такая быстрая) https://habr.com/ru/companies/okkamgroup/articles/509204/

import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings).astype("float32"))
faiss.write_index(index, "my_index.faiss")

In [51]:
print(index.ntotal) #количество индексов FAISS

4


In [56]:
topn = 2
D, I = index.search(embeddings[:3], topn)  # найдем 2 ближайших соседей для первых 3 векторов. Возвращает результат: Distances, Indices
print(I)
print(D)

[[0 1]
 [1 0]
 [2 3]]
[[0.         0.6635163 ]
 [0.         0.6635163 ]
 [0.         0.70921856]]


In [61]:
index1 = faiss.read_index("my_index.faiss") #Прочитаем векторную DB с индексами
index1
D, I = index1.search(embeddings[:3], topn)  # найдем 2 ближайших соседей для первых 3 векторов. Возвращает результат: Distances, Indices
print(I)


[[0 1]
 [1 0]
 [2 3]]


КЛАСТЕРИЗАЦИЯ в FAISS с помощью алгоритма k-means

В этом и состоит идея IVF индекса: сгруппируем большой набор векторов по частям с помощью алгоритма k-means, каждой части поставив в соответствие центроиду, – вектор, являющийся выбранным центром для данного кластера. Поиск будем осуществлять через минимальное расстояние до центроид, и только потом искать минимальные расстояния среди векторов в том кластере, что соответствует данной центроиде. Взяв k равным $\sqrt{n}$, где $n$ – количество векторов в индексе, мы получим оптимальный поиск на двух уровнях – сначала среди $\sqrt{n}$ центроид, затем среди $\sqrt{n}$ векторов в каждом кластере. Поиск по сравнению с полным перебором ускоряется в разы, что решает одну из наших проблем при работе с множеством миллионов векторов.


Пространство векторов разбивается методом k-means на k кластеров. Каждому кластеру в соответствие ставится центроида

Пример кода:


dim = 512
k = 1000  # количество “командиров”

quantiser = faiss.IndexFlatL2(dim)

index = faiss.IndexIVFFlat(quantiser, dim, k)

vectors = np.random.random((1000000, dim)).astype('float32')  # 1 000 000 “воинов”



А можно это записать куда более элегантно, воспользовавшись удобной штукой FAISS для построения индекса:

index = faiss.index_factory(dim, “IVF1000,Flat”)

Запускаем обучение:

print(index.is_trained)   # False.

index.train(vectors)  # Train на нашем наборе векторов


Обучение завершено, но векторов в индексе пока нет, так что добавляем их в индекс:
print(index.is_trained)  # True

print(index.ntotal)   # 0

index.add(vectors)

print(index.ntotal)   # 1000000



Рассмотрев такой тип индекса после Flat, мы решили одну из наших потенциальных проблем – скорость поиска, которая становится в разы меньше по сравнению с полным перебором.

In [36]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [50]:
#Запрос через агента ИИ

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_community.vectorstores import FAISS

vector_db = FAISS.load_local('my_index.faiss')

qa = RetrievalQA.from_chain_type(
    llm = OpenAI,
    chain_type = "staff",
    retriever = faiss.vector_db.as_retriever()
)
answer = qa.run("серега")

TypeError: FAISS.load_local() missing 1 required positional argument: 'embeddings'

In [10]:
from typing import Any, Dict, List, Literal, Optional
import torch
import langchain
#from config import settings
#from langchain_chroma import Chroma
from langchain_deepseek import ChatDeepSeek
#from langchain_openai import ChatOpenAI
#from langchain_huggingface import HuggingFaceEmbeddings
from loguru import logger

ModuleNotFoundError: No module named 'langchain_deepseek'

In [9]:
#Класс интегрировации моделей DeepSeek и ChatGPT в систему
class ChatWithAI:
    def __init__(self, provider: Literal["deepseek", "openai"] = "deepseek"):
        self.provider = provider
        self.embeddings = HuggingFaceEmbeddings(
            model_name=settings.LM_MODEL_NAME,
            model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
            encode_kwargs={"normalize_embeddings": True},
        )

        if provider == "deepseek":
            self.llm = ChatDeepSeek(
                api_key=settings.DEEPSEEK_API_KEY,
                model=settings.DEEPSEEK_MODEL_NAME,
                temperature=0.7,
            )
        elif provider == "openai":
            self.llm = ChatOpenAI(
                api_key=settings.OPENAI_API_KEY,
                model=settings.OPENAI_MODEL_NAME,
                temperature=0.7,
            )
        else:
            raise ValueError(f"Неподдерживаемый провайдер: {provider}")

        self.chroma_db = Chroma(
            persist_directory=settings.AMVERA_CHROMA_PATH,
            embedding_function=self.embeddings,
            collection_name=settings.AMVERA_COLLECTION_NAME,
        )

In [ ]:
#Этот метод трансформирует связку документ + метаданные из всех полученных документов в одно структурированное сообщение, которое мы затем передадим нейросети

def format_context(self, context: List[Dict[str, Any]]) -> str:
    """Форматирование контекста для промпта."""
    formatted_context = []
    for item in context:
        metadata_str = "\n".join(f"{k}: {v}" for k, v in item["metadata"].items())
        formatted_context.append(
            f"Текст: {item['text']}\nМетаданные:\n{metadata_str}\n"
        )
    return "\n---\n".join(formatted_context)